# Bathymetric particle filter 

This notebook is a testing and development environment for the geophysical particle filter. As of 30 December 2024 the initial implementation will be a velocity-state based implementation that does not do the proposed full state mechanization. The full state mechanization will be implemented in the future under a ROS2 and Gazebo simulation environment that allows for more realistic (and reliable) IMU simulation.

This notebook is meant for interactive testing and development and should not be used for large-scale simulations. The intended workflow is to use the notebook to test and develop the particle filter on a small subsample of test data, then build and install it in the local virtual environment for use in the full simulation.

This full simulation should then be written in the `/scripts` folder with result data saved off there.

## Simulation parameters verification

First we need to tune the particle filter propagation noise to be similar to that of a marine-grade inertial navigation system. A low-end marine-grade INS should have a drift of 1 nm per 24 hours. Our typical time interval is 60 seconds and the noise parameters for velocities states need to be tuned based on this value.

In [10]:
from src.geophysical.particle_filter import propagate_ned, rmse
import numpy as np


def run_velocity_loop(u: np.ndarray, noise: np.ndarray, dt: float = 60.0, time: int = 24 * 60) -> float:
    P = np.asarray([[0, 0, 0, u[0], u[1], u[2]]])
    T = P.copy()
    t = 0
    while t < time:
        P = propagate_ned(P, u, dt)  # noise=np.diag(noise), noise_calibration_mode=True)
        # Add zero-mean noise to the particles
        P += np.random.multivariate_normal(np.zeros(6), np.diag(noise))
        T = propagate_ned(T, u, dt)  # noise=np.diag([0, 0, 0]), noise_calibration_mode=False)
        t += 1
    error = rmse(P, T, include_altitude=True)
    return error

In [47]:
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
time = 24 * 60  # minutes
vel_noise = 1.75  # m/s
noise = np.array([0, 0, 0, vel_noise, vel_noise, 0.01])
bound = 1852  # meters
errors = []
errors_dict = {}
for i in range(1, 26):  # 1 to 25 nautical miles per hour
    print(f"Running for {i} knots")
    v = i * bound / 3600  # convert to m/s
    for a in tqdm(range(1000)):
        # Eastward
        u = np.asarray([0.0, v, 0.0])
        errors.append(run_velocity_loop(u, noise))
        # Northward
        u = np.asarray([v, 0, 0])
        errors.append(run_velocity_loop(u, noise))
        # Northeastward
        u = np.array([1, 1, 0]) / np.linalg.norm([1, 1, 0])
        u *= v
        errors.append(run_velocity_loop(u, noise))
    errors_dict[i] = errors.copy()
    print(f"Mean RMSE: {np.mean(errors): 0.2f} meters")
print(f"Overall RMSE: {np.mean(errors)}")
# Plot the previous

plt.figure(figsize=(12, 4))
plt.violinplot(errors_dict.values(), showmeans=True, showmedians=True)
plt.axhline(y=1852, color="k", linestyle="--", label="1 Nautical Mile")
plt.axhline(np.mean(errors), color="r", linestyle="-", label="Mean Error")
plt.xlabel("Velocity (knots)")
plt.ylabel("Error (meters)")
plt.title("Velocity Noise Tuning Error Distributions")
plt.legend(loc="upper left")
plt.savefig("velocity_noise_tuning.png")
plt.show()

Finalized noise model: $\begin{bmatrix} 0 & 0 & 0 & 1.75 & 1.75 & 0.01 \end{bmatrix}$

---------

## Develop measurement model

Next we need to develop the measurement value standard deviation. We'll first do some general examination of the data. Namely, investigating the sensor measurements to see if we can build a reasonable sensor model.

In [49]:
from src.data_management import dbmgr
db = dbmgr.DatabaseManager(".db")
summary = db.get_all_trajectories()
summary = summary.drop(summary.index[-1])
bathy_trajectories = summary[summary["depth"]]

In [ ]:
from src.geophysical import gmt_toolbox as gmt
from tqdm import tqdm

bathy_differences = np.empty((0,))

for id in tqdm(bathy_trajectories["id"]):
    data = db.get_trajectory(id)
    min_lon = data["lon"].min()
    max_lon = data["lon"].max()
    min_lat = data["lat"].min()
    max_lat = data["lat"].max()
    # min_lon, min_lat, max_lon, max_lat = gmt.inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    try:
        bathy_map = gmt.GeophysicalMap(
            gmt.MeasurementType.BATHYMETRY,
            gmt.ReliefResolution.FIFTEEN_SECONDS,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.25,
        )
    except AssertionError:
        if min_lat == max_lat:
            min_lat -= 0.25
            max_lat += 0.25
        if min_lon == max_lon:
            min_lon -= 0.25
            max_lon += 0.25
        bathy_map = gmt.GeophysicalMap(
            gmt.MeasurementType.BATHYMETRY,
            gmt.ReliefResolution.FIFTEEN_SECONDS,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.1,
        )
    except:
        print(f"Failed to get bathymetry map for {id}: {min_lon}, {max_lon}, {min_lat}, {max_lat}")
    # d_bathy = np.hstack([d_bathy, data["DEPTH"] - (-get_map_point(bathy_map, data.LON, data.LAT))])
    bathy_differences = np.append(
        bathy_differences, data["depth"] - (-bathy_map.get_map_point(data["lon"], data["lat"]))
    )

bathy_mean_d = np.mean(bathy_differences, where=~np.isnan(bathy_differences))
bathy_std = np.std(bathy_differences, where=~np.isnan(bathy_differences))

In [ ]:
from scipy.stats import mode


def plot_measurement_statistics(
    data: np.ndarray, title: str, xlabel: str, ylabel: str, filename: str, bin_count: int = 1000, xlims: tuple = None
):
    Mean = np.mean(data, where=~np.isnan(data))
    Std = np.std(data, where=~np.isnan(data))
    Median = np.median(data[~np.isnan(data)])
    Mode = mode(data[~np.isnan(data)]).mode
    plt.figure(figsize=(8, 4))
    plt.hist(data, bins=bin_count, density=True, alpha=0.75)
    plt.axvline(Mean, color="r", linestyle="--", label=f"Mean={Mean:0.2f}")
    plt.axvline(Mean + Std, color="k", linestyle="--", label=f"$\pm\sigma={Std:0.2f}$")
    plt.axvline(Mean - Std, color="k", linestyle="--")
    plt.axvline(Median, color="g", linestyle="--", label=f"Median={Median:0.2f}")
    plt.axvline(Mode, color="b", linestyle="--", label=f"Mode={Mode:0.2f}")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if xlims is not None:
        plt.xlim(xlims)
    plt.title(title)
    plt.legend(loc="upper left")
    plt.savefig(filename)
    plt.show()

Using the above, let's plot the data and see if we can build a reasonable sensor model.

In [ ]:
plot_measurement_statistics(
    bathy_differences,
    "Raw Bathymetry Measurement Statistics",
    "Difference (meters)",
    "Density",
    "bathy_diff_stats_raw.png",
    xlims=(-1000, 1000),
)
print(f"Max: {np.max(bathy_differences[~np.isnan(bathy_differences)])}")

So from the above we can see that the data resembles a normal distribution, but has some substantial outliers. So let's filter out the outliers and then plot the data again.

In [ ]:
# Filter out the outliers
bathy_differences_filtered = bathy_differences[np.abs(bathy_differences - bathy_mean_d) < 3 * bathy_std]
plot_measurement_statistics(
    bathy_differences_filtered,
    "Filtered Bathymetry Measurement Statistics",
    "Difference (meters)",
    "Density",
    "bathy_diff_stats_filtered.png",
    xlims=(-250, 250),
)

With the filtered data we can see that the data is much more normally distributed. We can now use this data to build a sensor model for the particle filter. This model is a simple zero-mean normal distribution with a standard deviation of 100 meters. The statistical mean of the data suggests that there might be a sensor bias, but do to the fact that bathymetric measurements provide a direct measurement of the vertical state, it is unclear if this is an error in the sensor or the state estimate.

--------------------

## Experimental configuration

There are a few questions that can be answered with this simulation:

1. As a general long-term navigation aide - akin to a GPS system - how well does the particle filter perform? This can be answered by running the particle filter with a well-known initial state and tracking the drift error over time.
2. How well does the particle filter perform in a short-term navigation scenario? In other words, given a de-localized INS, how well does the particle filter perform in recovering the true state? This scenario poses a more specific scenario where the platform needs a position fix from GPS but is unable to get one.

I'll analyze both scenarios in this notebook by using two different initial states. The first state will be a well-known state (akin to a GPS fix) and the second will be a de-localized state (delocalized to a drift rate ~1 nmi).

### Performance criteria

The performance of the particle filter will be evaluated based on the following haversine distance error criteria:
1. Overall weighted root-mean-square error of the entire particle cloud (a measure of the overall accuracy of the particle filter)
2. Haversine error of the particle filter estimate (weighted average of all particles)
3. Wasserstein distance between the particle filter estimate and the true state (a measure of the distribution of the particle filter estimate) where the covariance (variance along the diagonal) matrix is derived from the particle field.
These criteria should ultimately be used to evaluate the performance with respect to two concepts: the overall accuracy of the particle filter estimate to recover truth, and with respect to the distance traveled by the platform.

### Truth model

The source data used in this experiment is fundamentally survey data (i.e. measurements that are time stamped and geolocated). I simulate IMU data in order to develop a trajectory, but this isn't specifically recorded IMU data. In order to generate a seperate effective truth, the simulated IMU data is integrated to generate an INS trajectory. This integrated trajectory is then in need of correction. For a truth comparison, I then use the GPS locations of the geophysical measurements to correct the IMU integration to develop a "Truth" trajectory. The same integrated trajectory is then used (through the NED velocities) to propagate the particle filter. The particle filter itself then is corrected using the bathymetric measurements. The error metric are then a comparison of the particle filter's estimate to the INS's "truth" estimate.

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pyins import transform
from src.geophysical import gmt_toolbox as gmt

#####
from src.data_management import dbmgr
from src.geophysical import particle_filter as pf
from src.geophysical.gmt_toolbox import GeophysicalMap, MeasurementType, ReliefResolution
import haversine as hs
import h5py
from tqdm import tqdm


def save_simulation_results(
    filename: str, pf_result: pd.DataFrame, trajectory: pd.DataFrame, trajectory_sd: pd.DataFrame
):
    # Create an HDF5 file
    with h5py.File(f"{filename}.h5", "w") as f:
        # Save the result dataframe
        result_group = f.create_group("result")
        for column in pf_result.columns:
            result_group.create_dataset(column, data=pf_result[column].values)

        # Save the feedback.trajectory dataframe
        trajectory_group = f.create_group("trajectory")
        for column in trajectory.columns:
            trajectory_group.create_dataset(column, data=trajectory[column].values)

        # Save the feedback.trajectory_sd dataframe
        trajectory_sd_group = f.create_group("trajectory_sd")
        for column in trajectory_sd.columns:
            trajectory_sd_group.create_dataset(column, data=trajectory_sd[column].values)

In [2]:
db = dbmgr.DatabaseManager(".db")
all_trajs = db.get_all_trajectories()
pf_config = pf.ParticleFilterConfig.from_dict(
    {
        "n": 100,
        "cov": [0.01, 0.01, 1, 0.1, 0.1, 0.1, 0, 0, 0, 0],
        "noise": [0.0, 0.0, 0.0, 1.75, 1.75, 0.01, 0.001, 0.001, 0.001, 0.1],
        "input_config": pf.ParticleFilterInputConfig.VELOCITY,
        "measurement_config": [{"name": "bathymetry", "std": 100}],
    }
)

In [ ]:
bathy_trajectories = all_trajs[all_trajs["depth"]]
bathy_trajectories

In [ ]:
bathy_trajectories.loc[bathy_trajectories["id"] == 900]["duration"] <= 3600

In [11]:
valid_bathy_trajectories = bathy_trajectories[bathy_trajectories["duration"] >= 3600]

In [ ]:
valid_bathy_trajectories.tail()

In [ ]:
db.get_trajectory(108)

In [15]:
# Create the output directory for saving results
output_dir = "bathy_pf_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Loop over the data
for j in tqdm(bathy_trajectories["id"]):
    if bathy_trajectories.loc[bathy_trajectories.index[j], "duration"] < 3600:
        print(f"Trajectory {j} is too short: {all_trajs.iloc[j]['duration']}")
        continue
    name = db.get_all_trajectories()["source"][j] + "_" + str(j)

    try:
        truth = db.get_trajectory(j)
    except:
        print(f"Failed to get trajectory {j}")
        continue
    try:
        _, feedback = pf.calculate_truth(truth)
        trajectory = feedback.trajectory
        ins_errors = transform.compute_state_difference(truth, trajectory)
    except:
        print(f"Failed to calculate truth for trajectory {j}")
        continue
    distance = truth["distance"].to_numpy()

    min_lat = truth.lat.min()
    max_lat = truth.lat.max()
    min_lon = truth.lon.min()
    max_lon = truth.lon.max()

    try:
        bathy_map = GeophysicalMap(
            MeasurementType.BATHYMETRY, ReliefResolution.FIFTEEN_SECONDS, min_lon, max_lon, min_lat, max_lat, 0.25
        )
    except:
        print(f"Failed to get bathymetry map for trajectory {j}")
        continue
    geomaps = {gmt.MeasurementType.BATHYMETRY: bathy_map}
    try:
        result = pf.run_particle_filter(truth, trajectory, geomaps, pf_config)
    except:
        print(f"Failed to run particle filter for trajectory {j}")
        continue
    try:
        out_path = os.path.join(output_dir, f"{name}")
        save_simulation_results(f"{out_path}", result, trajectory, feedback.trajectory_sd)
    except:
        print(f"Failed to save results for trajectory {j}")
        continue

In [ ]:
plt.contourf(bathy_map.map_data.lon, bathy_map.map_data.lat, bathy_map.map_data, cmap="ocean")
plt.plot(truth.lon, truth.lat, "k.", label="Truth")
plt.plot(trajectory.lon, trajectory.lat, "g.", label="INS")
plt.plot(result.lon, result.lat, "y.", label="PF")
plt.colorbar()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("2D trajectory")
# plt.xlim(min_lon, max_lon)
# plt.ylim(min_lat, max_lat)
plt.legend()
plt.show()

In [ ]:
hsv = hs.haversine_vector(truth[["lat", "lon"]].to_numpy(), result[["lat", "lon"]].to_numpy(), hs.Unit.METERS)
plt.plot(result.index, result["rms_error_2d"], "b")
plt.plot(result.index, hsv, "g")
plt.plot(result.index, result["estimate_error"], "y")
plt.ylim(0, 3000)
plt.axhline(1852)
plt.axhline(452, color="r")
plt.show()

In [364]:
summary = pd.DataFrame()
error_summary = pd.Series(err, distance)
summary[f"{j}"] = error_summary

In [ ]:
plt.plot(distance, estimate[:, 3:6])

In [ ]:
# Plot the results as a function of distance traveled
plt.plot(
    distance / 1852,
    100 * np.sqrt(ins_errors.north**2 + ins_errors.east**2) / distance,
    "g-",
    label="INS error",
)
plt.plot(
    distance / 1852,
    100 * np.sqrt(integrator_error.north**2 + integrator_error.east**2) / distance,
    "r.",
    label="integrator error",
)
plt.xlabel("Distance traveled (nmi)")
plt.ylabel("Position error (% of distance traveled)")
plt.title("Radial position error vs. distance traveled")
plt.legend()
plt.ylim(0, 100)
plt.show()

# Data Post Processing

Use this section to load and post process the results data set

In [1]:
import os
import numpy as np
import pandas as pd
import haversine as hs
from matplotlib import pyplot as plt
from glob import glob
import h5py
from src.geophysical import gmt_toolbox as gmt
from src.data_management.m77t import find_periods
from scripts import open_loop as ol
from datetime import datetime, timedelta

In [2]:
result_files = glob(os.path.join("scripts", "bathy_pf_results_delocalized", "*.h5"))

In [3]:
ol.summarize_results("scripts/bathy_pf_results", "delocalized")
ol.summarize_results("scripts/bathy_pf_results", "localized")

The goal of this experiment is to test the particle filter's ability to estimate the true state, either by maintaining an originally accurate estimate, or by recovering from a de-localized state. We are simulating a marine-grade INS with a drift rate of 1 nm per 24 hours, but comparing and propagting this sytem to a true INS value with GPS measurements.

Error metrics we are interested in:

1. Raw particle filter estimate error in meters plus the standard deviation of the particle filter estimate (a measure of overall accuracy and confidence)
2. Particle filter error normalized by the distance traveled by the platform (a measure of the relative error)
3. Particle filter error normalized by drift rate (indicates how much of the error is due to the drift rate, and how much is due to the particle filter) which should demonstrate it's suitability as a long-term navigation aide for position feedback.
4. Particle filter error (possibly normalized as above) as a percentage of map pixel resolution (15 arcseconds, ~452 meters).

Key finding that we want to prove is that the particle filter can estimate the true state to within the resolution of a given map pixel. This is a key finding as it demonstrates the particle filter's ability to provide a position fix in the absence of GPS that is better than the resolution of the map. Alternatively, if we had an arbitrary map where each pixel had a completely unique signature, the measurement model would only be able to report the particle filter's estimate to within the resolution of the map. Subsequent movements and measurments could then be used to refine the estimate. Because geophysical maps and measurements are not unique, the ability of the particle filter to estimate the true state to within the resolution of the map is a key finding that indicates a performance level that approaches a theoretical limit.

### General idea

Consider a grid based map $m$ with entirely unique signatures $m = \left[m_0 \dots m_n\right]$ where $m_i \neq m_j$ for all $i \neq j$. We have a means to measure that signature with perfect accuracy maing the measurement model $p(z | m) = 1$ for $z = m_i$, and zero elsewhere. $\text{bel}(x_t) = \text{bel}(x_{t-1}) * p(z | m)$. To derive a navigation fix we would then take the maximum value of $\text{bel}(x_t)$ and use that as the navigation fix. However, without knowledge of motion, the state space is similarly limited to the grid cell size. With knowledge of motion, this measurement function becomes a lower limit to the amount of error the position estimate can be resolved to.

# Gravity and Magnetics

Recreate the above measurment model development this time with gravity freeair anomaly and magnetic anomaly data.

In [25]:
from src.data_management import dbmgr

db = dbmgr.DatabaseManager(".db")
summary = db.get_all_trajectories()
summary = summary.drop(summary.index[-1])
grav_trajectories = summary[summary["freeair"]]
mag_trajectories = summary[summary["mag_res"]]

In [ ]:
from src.geophysical import gmt_toolbox as gmt
from tqdm import tqdm

grav_differences = np.empty((0,))

for id in tqdm(grav_trajectories["id"]):
    data = db.get_trajectory(id)
    min_lon = data["lon"].min()
    max_lon = data["lon"].max()
    min_lat = data["lat"].min()
    max_lat = data["lat"].max()
    # min_lon, min_lat, max_lon, max_lat = gmt.inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    try:
        gravity_map = gmt.GeophysicalMap(
            gmt.MeasurementType.GRAVITY,
            gmt.GravityResolution.ONE_MINUTE,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.25,
        )
    except AssertionError:
        if min_lat == max_lat:
            min_lat -= 0.25
            max_lat += 0.25
        if min_lon == max_lon:
            min_lon -= 0.25
            max_lon += 0.25
        gravity_map = gmt.GeophysicalMap(
            gmt.MeasurementType.GRAVITY,
            gmt.GravityResolution.ONE_MINUTE,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.1,
        )
    except Exception as e:
        print(f"Failed to get gravity map for {id}: {min_lon}, {max_lon}, {min_lat}, {max_lat}")
        print(e)
    # d_bathy = np.hstack([d_bathy, data["DEPTH"] - (-get_map_point(bathy_map, data.LON, data.LAT))])
    grav_differences = np.append(
        grav_differences, data["freeair"] - (gravity_map.get_map_point(data["lon"], data["lat"]))
    )

gravity_mean_d = np.mean(grav_differences, where=~np.isnan(grav_differences))
gravity_std = np.std(grav_differences, where=~np.isnan(grav_differences))

In [ ]:
plot_measurement_statistics(
    grav_differences,
    "Raw Gravity Measurement Statistics",
    "Difference (mgal)",
    "Density",
    "gravity_diff_stats_raw.png",
    xlims=(-50, 100),
)
print(f"Max: {np.max(grav_differences[~np.isnan(grav_differences)])}")

In [ ]:
mag_differences = np.empty((0,))

for id in tqdm(grav_trajectories["id"]):
    data = db.get_trajectory(id)
    min_lon = data["lon"].min()
    max_lon = data["lon"].max()
    min_lat = data["lat"].min()
    max_lat = data["lat"].max()
    # min_lon, min_lat, max_lon, max_lat = gmt.inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    try:
        mag_map = gmt.GeophysicalMap(
            gmt.MeasurementType.MAGNETIC,
            gmt.MagneticResolution.TWO_MINUTES,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.25,
        )
    except AssertionError:
        if min_lat == max_lat:
            min_lat -= 0.25
            max_lat += 0.25
        if min_lon == max_lon:
            min_lon -= 0.25
            max_lon += 0.25
        mag_map = gmt.GeophysicalMap(
            gmt.MeasurementType.MAGNETIC,
            gmt.MagneticResolution.TWO_MINUTES,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.1,
        )
    except Exception as e:
        print(f"Failed to get magnetic map for {id}: {min_lon}, {max_lon}, {min_lat}, {max_lat}")
        print(e)
    # d_bathy = np.hstack([d_bathy, data["DEPTH"] - (-get_map_point(bathy_map, data.LON, data.LAT))])
    mag_differences = np.append(
        mag_differences, data["mag_res"] - (mag_map.get_map_point(data["lon"], data["lat"]))
    )

mag_mean_d = np.mean(mag_differences, where=~np.isnan(mag_differences))
mag_std = np.std(mag_differences, where=~np.isnan(mag_differences))

In [ ]:
plot_measurement_statistics(
    mag_differences,
    "Raw Magnetic Measurement Statistics",
    "Difference (mgal)",
    "Density",
    "magnetics_diff_stats_raw.png",
    xlims=(-1000, 1000),
)
print(f"Max: {np.max(mag_differences[~np.isnan(mag_differences)])}")

In [ ]:
print(f"Gravity Mean: {gravity_mean_d}, Gravity Std: {gravity_std}")
print(f"Magnetic Mean: {mag_mean_d}, Magnetic Std: {mag_std}")

In [36]:
import h5py

In [37]:
anomaly_data = {
    "gravity_differences" : grav_differences,
    "magnetic_differences" : mag_differences
}

with h5py.File("anomaly_data.h5", "w") as f:
    for key in anomaly_data:
        f.create_dataset(key, data=anomaly_data[key])

## Some post processing

In [1]:
import pandas as pd
from scripts import open_loop as ol
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [111]:
initialization = "delocalized"
offset = 1852

In [112]:
estimate_error_summary_distance = pd.read_csv(f"scripts/bathy_pf_results/{initialization}/estimate_error_summary_distance.csv", index_col=0)
estimate_error_summary_time = pd.read_csv(f"scripts/bathy_pf_results/{initialization}/estimate_error_summary_time.csv", index_col=0)
time = estimate_error_summary_time.index.to_numpy()
drift = time * (1852 / (24*3600)) + offset
mean = estimate_error_summary_time.mean(axis=1)#.rolling(window=1000, min_periods=1).mean().to_numpy()
median = estimate_error_summary_time.median(axis=1)#.rolling(window=1000, min_periods=1).median().to_numpy()
std = estimate_error_summary_time.std(axis=1)#.rolling(window=100, min_periods=1).std().to_numpy()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 2))
ax.plot(time / 3600, mean / drift, '-b')
ax.fill_between(time / 3600, (mean - std) / drift, (mean + std) / drift, color='b', alpha=0.25)
ax.set_xlim(0, 150)
ax.set_ylim(0, 100)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Multiple of Drift")
ax.set_title(f"{initialization} Particle Filter Mean Percent Error Over Time".title())
ax.legend(["Mean", "Standard Deviation"])
plt.savefig(f"{initialization}_pf_error_time.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 2))
ax.plot(time / 3600, median / drift, '-g')
ax.fill_between(time / 3600, (median - std) / drift, (median + std) / drift, color='g', alpha=0.25)
ax.set_xlim(0, 150)
ax.set_ylim(0, 100)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Multiple of Drift")
ax.set_title(f"{initialization} Particle Filter Median Percent Error Over Time".title())
ax.legend(["Median", "Standard Deviation"])
plt.savefig(f"{initialization}_pf_median_error_time.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 2))
ax.plot(time / 3600, mean / drift, '-b')
ax.plot(time / 3600, median / drift, '-g')
ax.set_xlim(0, 24)
ax.set_ylim(0, 15)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Multiple of Drift")
ax.set_title(f"{initialization} Particle Filter Percent Error Over Time (Zoomed)".title())
ax.axhline(1, color='m', linestyle='--', label="drift")
ax.legend(["Mean", "Median", "Drift"])
plt.savefig(f"{initialization}_pf_error_time_zoomed.png")
plt.show()

In [117]:
bp = pd.read_csv(f"scripts/bathy_pf_results/{initialization}/below_pixel.csv", index_col=0)

## Magnetics Testing

In [1]:
from scripts import open_loop_mag as olm
from src.data_management import dbmgr
from src.geophysical import gmt_toolbox as gmt
from src.geophysical import particle_filter as pf
import numpy as np

In [2]:
db = dbmgr.DatabaseManager(".db")
all_trajs = db.get_all_trajectories()
mag_trajectories = all_trajs.loc[(all_trajs['duration'] >= 3600) & (all_trajs["mag_res"])]
truth = db.get_trajectory(75)
_, feedback = pf.calculate_truth(truth)
trajectory = feedback.trajectory
min_lat = truth.lat.min()
max_lat = truth.lat.max()
min_lon = truth.lon.min()
max_lon = truth.lon.max()
mag_map = gmt.GeophysicalMap(
        gmt.MeasurementType.MAGNETIC, gmt.MagneticResolution.TWO_MINUTES, min_lon, max_lon, min_lat, max_lat, 0.25
    )
geomaps = {gmt.MeasurementType.MAGNETIC: mag_map}

In [3]:
pf_config_localized = pf.ParticleFilterConfig.from_dict(
    {
        "n": 10,
        "cov": [15 / (1852 * 60), 15 / (1852 * 60), 1, 0.1, 0.1, 0.1, 0, 0, 0, 0],
        "noise": [0.0, 0.0, 0.0, 1.75, 1.75, 0.01, 0.001, 0.001, 0.001, 0.1],
        "input_config": pf.ParticleFilterInputConfig.VELOCITY,
        "measurement_config": [{"name": "magnetic", "std": 194}],
    }
)

In [4]:
pf_config_localized.get_base_state()

['lat', 'lon', 'alt', 'VN', 'VE', 'VD']

In [5]:
pf_config_localized.measurement_config[0].name == gmt.MeasurementType.MAGNETIC

True

In [6]:
olm.pf_config_delocalized.measurement_config[0].name == gmt.MeasurementType.BATHYMETRY

False

In [7]:
pf.GeophysicalMeasurement.from_dict({'name': 'magnetic', 'std': 100})

<MAGNETIC>:100

In [8]:
particles = pf.initialize_particle_filter(
    truth.loc[truth.index[0], pf_config_localized.get_base_state()].to_numpy(), pf_config_localized)
weights = np.ones((pf_config_localized.n,)) / pf_config_localized.n
    

In [9]:
w = pf.update_anomaly(
    particles,
    geomaps[pf_config_localized.measurement_config[0].name],
    truth.loc[truth.index[2], "mag_res"],
    pf_config_localized.measurement_config[0].std,
    particles[:, 9],
)

In [10]:
w

array([0.00179134, 0.00193675, 0.00182548, 0.00183631, 0.00186271,
       0.001942  , 0.00200364, 0.00188747, 0.0019346 , 0.00190755])

In [11]:
result = pf.run_particle_filter(truth, trajectory, geomaps, pf_config_localized)

In [12]:
result

,lat,lon,alt,vn,ve,vd,lat_var,lon_var,alt_var,vn_var,ve_var,vd_var,estimate_error,rms_error_2d,rms_error_3d
0.0,50.903175,-16.115902,-0.112497,-5.012516,2.666637,-0.027767,5.541946e-05,1.197921e-04,0.272704,0.266718,0.070401,0.023402,271.530422,367.145779,367.145818
60.0,50.897779,-16.107291,1.314101,-3.883884,2.434596,0.020797,8.188542e-05,3.416904e-05,17.183104,3.139431,4.840458,0.011973,529.696568,382.237005,382.239479
120.0,50.898925,-16.099626,3.827372,-7.008722,0.349822,-0.041564,3.128263e-05,1.371918e-06,25.394950,0.743474,2.415625,0.059416,622.000857,279.361390,279.368557
180.0,50.902243,-16.096386,-7.168154,-0.931166,4.644753,-0.049855,1.777557e-05,4.340294e-06,10.058537,2.680364,3.030161,0.013048,744.482814,282.027382,282.038275
240.0,50.901890,-16.091699,-10.485224,-1.737480,3.663537,0.072032,1.539116e-05,2.738825e-06,36.252704,4.552998,2.190967,0.013349,909.505160,321.089199,321.111963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7740.0,50.679352,-15.798395,2962.178737,-4.712367,2.263704,-0.687988,7.473236e-07,1.757772e-06,90.414433,2.490380,0.963817,0.013990,7113.193913,2249.789217,2437.008586
7800.0,50.677234,-15.796613,3016.516113,-0.789704,4.772731,-0.734587,4.928823e-07,5.281441e-09,37.013352,0.582241,2.161861,0.009559,7176.870937,2269.660710,2461.970834
7860.0,50.675882,-15.793824,3061.337866,-5.216946,2.700591,-0.804873,5.061716e-07,4.292024e-07,37.873667,2.396877,8.578416,0.027875,7331.937784,2318.743089,2512.718101
7920.0,50.674132,-15.792997,3105.051552,-4.247424,1.825434,-0.682056,7.865278e-07,2.979142e-06,48.602451,1.056059,6.695859,0.007635,7570.920911,2394.647055,2588.141011


In [ ]:
truth.head()